In [6]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pandas as pd

import pandas as pd
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
from torchvision import transforms, models

In [26]:
np.random.seed(54321)

In [11]:
class DigitsDataset(Dataset):
    def __init__(self, data, transform, attributes, classes_subset, shuffle = True):
        self.data = data[data.label.isin(classes_subset)].sample(frac=1).reset_index(drop=True) if shuffle else data
        self.transform = transform
        self.attributes = attributes

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data.iloc[index]
        pixels = np.array(item[1:]).astype(np.uint8).reshape((28, 28))
        label = item[0]
        attributes = self.attributes[self.attributes.Digit == int(label)].iloc[:, 1:].values
        if self.transform is not None:
            pixels = self.transform(pixels)

        return pixels, label, attributes

In [4]:
class CNN(nn.Module):
    def __init__(self, alpha, output_dim, batch_size):
        super().__init__()
        self.batch_size = batch_size
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5)
        self.lin = nn.Linear(20 * 20 * 32, output_dim)


        self.loss = nn.BCELoss()
        self.optimizer = torch.optim.SGD(self.parameters(), lr = alpha)

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(self.device)
        self.to(self.device)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.lin(x.view(-1, 20 * 20 * 32))
        x = F.sigmoid(x)
        return x

    def save(self, path):
        T.save(self.state_dict(), path)

    def load(self, path):
        self.load_state_dict(T.load(path))

In [7]:
def get_euclidean_dist(curr_labels, class_labels):
    return np.sqrt(np.sum((curr_labels - class_labels) ** 2))

def predict_label(predicted_attributes, all_attributes, classes_to_predict_from):
    predicted_labels = []
    attribute_subset = all_attributes[all_attributes.Digit.isin(classes_to_predict_from)]
    for predicted_attribute in predicted_attributes:
        distances = [(class_label, get_euclidean_dist(predicted_attribute, attribute_subset[attribute_subset.Digit == class_label].iloc[:,1:].values)) for class_label in classes_to_predict_from]
        predicted_label = sorted(distances, key = lambda x: x[1])[0][0]
        predicted_labels.append(predicted_label)
    return np.array(predicted_labels)

In [34]:
batch_size = 32
n_epochs = 30
learning_rate = 0.05

transformation = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean = (0.5, ), std = (0.5, ))
])

In [28]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
attributes_df = pd.read_csv("../data/mnist_attributes.csv")

params = {'batch_size': batch_size, 'shuffle': True, 'num_workers': 6}

train_split = 0.7
n_rows = len(train)
len_training = int(n_rows * train_split)

training_classes = [0,1,2,3,4,5]
testing_classes = [6,7,8,9]

train_indexes = np.random.choice(len(train), size = len_training, replace = False)

train_dataset = DigitsDataset(train.iloc[train_indexes], attributes = attributes_df, transform = transformation, classes_subset=training_classes)
training_generator = DataLoader(train_dataset, **params)

validation_dataset = DigitsDataset(train.iloc[~train_indexes], attributes = attributes_df, transform = transformation, classes_subset=testing_classes)
validation_generator = DataLoader(validation_dataset, **params)



cuda


In [35]:
model = CNN(alpha = learning_rate, batch_size = batch_size, output_dim=7)
model.train()
for i in range(n_epochs+1):
    losses = []
    for images, labels, attributes in training_generator:
        images = images.to(model.device)
        labels = labels.to(model.device)
        attributes = attributes.view(-1, 7).float().to(model.device)
        model.optimizer.zero_grad()
        attributes_predicted = model.forward(images).float()
        loss = model.loss(attributes_predicted, attributes).to(model.device)
        losses.append(loss.item())
        loss.backward()
        model.optimizer.step()
    print(f"Epoch: {i}, Loss: {np.mean(losses)}")
    if i%10 == 0:
        with T.no_grad():
            accuracy = []
            for images, labels, attributes in validation_generator:
                images = images.to(model.device)
                labels = labels
                attributes_predicted = model.forward(images).float()
                predicted_labels = predict_label(attributes_predicted.cpu().numpy(), attributes_df, classes_to_predict_from=testing_classes)
                n_correct_attributes =  np.sum(labels.numpy() == predicted_labels)
                accuracy.append(n_correct_attributes * 100.0 / len(labels))
            print(f"Epoch: {i}, Accuracy: {np.mean(accuracy)}")

cuda
Epoch: 0, Loss: 0.12673683821056947
Epoch: 0, Accuracy: 42.915531335149865
Epoch: 1, Loss: 0.03791871742651904
Epoch: 2, Loss: 0.02816648607581827
Epoch: 3, Loss: 0.023542314281335158
Epoch: 4, Loss: 0.02095813926913107
Epoch: 5, Loss: 0.01855555548806082
Epoch: 6, Loss: 0.01673741827566921
Epoch: 7, Loss: 0.015358390629495313
Epoch: 8, Loss: 0.014279267854412182
Epoch: 9, Loss: 0.013406086840595676
Epoch: 10, Loss: 0.012495416700423175
Epoch: 10, Accuracy: 50.49386920980926
Epoch: 11, Loss: 0.011205122276078986
Epoch: 12, Loss: 0.010504062380873267
Epoch: 13, Loss: 0.009804484412111139
Epoch: 14, Loss: 0.009482488401190363
Epoch: 15, Loss: 0.008552182829326438
Epoch: 16, Loss: 0.008084309924285266
Epoch: 17, Loss: 0.007699286712993107
Epoch: 18, Loss: 0.006922756196708068
Epoch: 19, Loss: 0.00673982410487057
Epoch: 20, Loss: 0.006381511355498802
Epoch: 20, Accuracy: 50.366144414168936
Epoch: 21, Loss: 0.005947066550562253
Epoch: 22, Loss: 0.005345270055360717
Epoch: 23, Loss: 0.0